In [1]:
from IPython.core.display import display
from datautils.data_utils import get_dev_data, merge_trades_and_quotes
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
data = get_dev_data()
data = merge_trades_and_quotes(data)

In [3]:
df = data[0]
display(df.tail(5))

,SYM,DATE_TIME,ASK_PRICE,ASK_SIZE,BID_PRICE,BID_SIZE,SIZE,PRICE
57475,XLE,2012-02-01 15:29:49,71.18,13,71.17,23,125,71.17
57476,XLE,2012-02-01 15:29:51,71.18,11,71.17,20,125,71.17
57477,XLE,2012-02-01 15:29:53,71.18,17,71.17,21,125,71.17
57478,XLE,2012-02-01 15:29:54,71.18,18,71.17,14,125,71.17
57479,XLE,2012-02-01 15:29:57,71.18,17,71.17,12,125,71.17


# Rolling volume sum

In [4]:
def add_rolling_trade_sum(data, window):
    start_dates = data['DATE_TIME'] - window
    data['start_index'] = data['DATE_TIME'].values.searchsorted(start_dates, side='right')
    data['end_index'] = np.arange(len(data))
    def sum_window(row):
        return data['SIZE'].iloc[row['start_index']:row['end_index']+1].sum()
    data['TRADE_SUM'] = data.apply(sum_window, axis=1)
    data = data.drop(['start_index', 'end_index'], axis=1, inplace=True)

In [5]:
window = pd.Timedelta(minutes=1)
add_rolling_trade_sum(df, window)
display(df.tail(10))

,SYM,DATE_TIME,ASK_PRICE,ASK_SIZE,BID_PRICE,BID_SIZE,SIZE,PRICE,TRADE_SUM
57470,XLE,2012-02-01 15:29:44,71.18,10,71.17,22,125,71.17,23229
57471,XLE,2012-02-01 15:29:45,71.18,12,71.17,22,125,71.17,23254
57472,XLE,2012-02-01 15:29:46,71.18,16,71.17,20,125,71.17,23279
57473,XLE,2012-02-01 15:29:47,71.18,12,71.17,20,125,71.17,23304
57474,XLE,2012-02-01 15:29:48,71.18,16,71.17,18,125,71.17,23429
57475,XLE,2012-02-01 15:29:49,71.18,13,71.17,23,125,71.17,23554
57476,XLE,2012-02-01 15:29:51,71.18,11,71.17,20,125,71.17,23079
57477,XLE,2012-02-01 15:29:53,71.18,17,71.17,21,125,71.17,23104
57478,XLE,2012-02-01 15:29:54,71.18,18,71.17,14,125,71.17,23229
57479,XLE,2012-02-01 15:29:57,71.18,17,71.17,12,125,71.17,23154


In [6]:
display(df.head(5))

,SYM,DATE_TIME,ASK_PRICE,ASK_SIZE,BID_PRICE,BID_SIZE,SIZE,PRICE,TRADE_SUM
0,XLE,2012-02-01 09:30:00.000,71.385455,17,71.35,19,NaN,NaN,0
1,XLE,2012-02-01 09:30:00.012,71.385455,17,71.35,19,286,71.32,286
2,XLE,2012-02-01 09:30:00.012,71.385455,17,71.35,19,417,71.33,703
3,XLE,2012-02-01 09:30:00.030,71.385455,17,71.35,19,100,71.33,803
4,XLE,2012-02-01 09:30:00.030,71.385455,17,71.35,19,100,71.33,903


# Hits&Lifts

In [7]:
def add_vpin_time(data, window):
    def lifts(row):
        lift = -1
        if row['PRICE'] == row['ASK_PRICE']:
            lift = 1
        return row['SIZE']*lift
    data['lift'] = data.apply(lifts, axis=1)
    start_dates = data['DATE_TIME'] - window
    data['start_index'] = data['DATE_TIME'].values.searchsorted(start_dates, side='right')
    data['end_index'] = np.arange(len(data))
    def sum_window(row):
        return data['SIZE'].iloc[row['start_index']:row['end_index']+1].sum()
    def sum_lifts(row):
        return data['lift'].iloc[row['start_index']:row['end_index']+1].sum()
    data['trade_sum'] = data.apply(sum_window, axis=1)
    data['lift_sum'] = data.apply(sum_lifts, axis=1)
    data['VPIN_TIME'] = data['lift_sum']/data['trade_sum']
    data = data.drop(['start_index', 'end_index', 'lift', 'trade_sum', 'lift_sum'], axis=1, inplace=True)

In [8]:
window = pd.Timedelta(minutes=1)
add_vpin_time(df, window)
display(df.tail(5))

,SYM,DATE_TIME,ASK_PRICE,ASK_SIZE,BID_PRICE,BID_SIZE,SIZE,PRICE,TRADE_SUM,VPIN_TIME
57475,XLE,2012-02-01 15:29:49,71.18,13,71.17,23,125,71.17,23554,-0.906598
57476,XLE,2012-02-01 15:29:51,71.18,11,71.17,20,125,71.17,23079,-0.904675
57477,XLE,2012-02-01 15:29:53,71.18,17,71.17,21,125,71.17,23104,-0.904778
57478,XLE,2012-02-01 15:29:54,71.18,18,71.17,14,125,71.17,23229,-0.905291
57479,XLE,2012-02-01 15:29:57,71.18,17,71.17,12,125,71.17,23154,-0.904984
